In [1]:
from narwhals import read_csv

from src.data.load_data import load_and_prepare
from src.data.preprocess import preprocess_load_metric

In [8]:
df_raw = load_and_prepare("~/Code/ntels_Project/src/data/node_metrics.csv")

In [9]:
df_raw

,metric,value,timestamp,cpu,mode,device,device_error,fstype,mountpoint
0,node_cpu_seconds_total,5.029281e+04,2025-07-06 17:30:01,14,system,NaN,NaN,NaN,NaN
1,node_cpu_seconds_total,2.637569e+05,2025-07-06 17:30:01,14,user,NaN,NaN,NaN,NaN
2,node_cpu_seconds_total,2.346515e+06,2025-07-06 17:30:01,15,idle,NaN,NaN,NaN,NaN
3,node_cpu_seconds_total,4.934523e+04,2025-07-06 17:30:01,15,iowait,NaN,NaN,NaN,NaN
4,node_cpu_seconds_total,0.000000e+00,2025-07-06 17:30:01,15,irq,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4327651,node_time_seconds,1.753604e+09,2025-07-27 08:15:02,NaN,NaN,NaN,NaN,NaN,NaN
4327652,node_time_seconds,1.753604e+09,2025-07-27 08:20:02,NaN,NaN,NaN,NaN,NaN,NaN
4327653,node_time_seconds,1.753605e+09,2025-07-27 08:25:02,NaN,NaN,NaN,NaN,NaN,NaN
4327654,node_time_seconds,1.753605e+09,2025-07-27 08:30:02,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd

df_processed = pd.read_csv("best_model_preprocessed.csv")
df_processed['mean'].describe()

count    11870.000000
mean         0.976459
std          2.334378
min          0.140000
25%          0.445000
50%          0.550000
75%          0.755000
max         29.715000
Name: mean, dtype: float64

In [66]:
df_raw.loc[df_raw["metric"] == "node_load1"]

,metric,value,timestamp,cpu,mode,device,device_error,fstype,mountpoint
1739124,node_load1,0.42,2025-07-20 01:00:01,NaN,NaN,NaN,NaN,NaN,NaN
1739125,node_load1,0.64,2025-07-20 01:00:01,NaN,NaN,NaN,NaN,NaN,NaN
1739126,node_load1,0.30,2025-07-20 01:05:02,NaN,NaN,NaN,NaN,NaN,NaN
1739127,node_load1,0.61,2025-07-20 01:05:02,NaN,NaN,NaN,NaN,NaN,NaN
1739128,node_load1,0.33,2025-07-20 01:10:02,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3853238,node_load1,0.58,2025-07-20 00:45:02,NaN,NaN,NaN,NaN,NaN,NaN
3853239,node_load1,0.18,2025-07-20 00:50:02,NaN,NaN,NaN,NaN,NaN,NaN
3853240,node_load1,0.54,2025-07-20 00:50:02,NaN,NaN,NaN,NaN,NaN,NaN
3853241,node_load1,0.29,2025-07-20 00:55:01,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_preprocessed = preprocess_load_metric(df_raw)

In [11]:
df_preprocessed

,mean,max,min,std
timestamp,,,,
2025-06-17 00:45:00,11.450,13.14,9.76,2.390021
2025-06-17 00:50:00,14.905,15.91,13.90,1.421285
2025-06-17 00:55:00,14.645,14.90,14.39,0.360624
2025-06-17 01:00:00,13.380,13.99,12.77,0.862670
2025-06-17 01:05:00,12.735,13.51,11.96,1.096016
...,...,...,...,...
2025-07-28 05:30:00,2.075,2.85,1.30,1.096016
2025-07-28 05:35:00,0.765,1.11,0.42,0.487904
2025-07-28 05:40:00,0.785,0.99,0.58,0.289914


In [14]:
from src.features.feature_engineering import make_features
print("🛠 피처 엔지니어링 실행...")
df_features = make_features(
    df_preprocessed,
    lags=[1, 2, 3],       # 1~3 스텝 전 값
    rolling_window=[3, 6], # 3개/6개 이동평균
    use_all_stats=True,
    use_holidays=True,
    use_fourier=True
)

print("✅ 피처 shape:", df_features.shape)
df_features.head(10)

🛠 피처 엔지니어링 실행...
✅ 피처 shape: (11865, 35)


,mean,max,min,std,mean_lag_1,mean_lag_2,mean_lag_3,mean_rolling_mean_3,mean_rolling_mean_6,max_lag_1,...,weekday,month,is_weekend,is_holiday,fft_real_1,fft_imag_1,fft_real_2,fft_imag_2,fft_real_3,fft_imag_3
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-06-17 01:10:00,14.895,15.88,13.91,1.393000,12.735,13.380,14.645,13.670000,13.668333,13.51,...,1,6,0,0,0.999912,-0.013233,0.999912,0.013233,1.0,0.0
2025-06-17 01:15:00,13.395,13.82,12.97,0.601041,14.895,12.735,13.380,13.675000,13.992500,15.88,...,1,6,0,0,0.999874,-0.015879,0.999874,0.015879,1.0,0.0
2025-06-17 01:20:00,7.095,11.49,2.70,6.215469,13.395,14.895,12.735,11.795000,12.690833,13.82,...,1,6,0,0,0.999828,-0.018526,0.999828,0.018526,1.0,0.0
2025-06-17 01:25:00,4.415,8.43,0.40,5.678067,7.095,13.395,14.895,8.301667,10.985833,11.49,...,1,6,0,0,0.999776,-0.021172,0.999776,0.021172,1.0,0.0
2025-06-17 01:30:00,3.225,6.14,0.31,4.122433,4.415,7.095,13.395,4.911667,9.293333,8.43,...,1,6,0,0,0.999716,-0.023818,0.999716,0.023818,1.0,0.0
2025-06-17 01:35:00,2.410,4.52,0.30,2.983991,3.225,4.415,7.095,3.350000,7.572500,6.14,...,1,6,0,0,0.999650,-0.026464,0.999650,0.026464,1.0,0.0
2025-06-17 01:40:00,2.005,3.41,0.60,1.986970,2.410,3.225,4.415,2.546667,5.424167,4.52,...,1,6,0,0,0.999576,-0.029109,0.999576,0.029109,1.0,0.0
2025-06-17 01:45:00,2.410,2.69,2.13,0.395980,2.005,2.410,3.225,2.275000,3.593333,3.41,...,1,6,0,0,0.999496,-0.031755,0.999496,0.031755,1.0,0.0
2025-06-17 01:50:00,1.075,1.99,0.16,1.294005,2.410,2.005,2.410,1.830000,2.590000,2.69,...,1,6,0,0,0.999408,-0.034400,0.999408,0.034400,1.0,0.0


In [26]:
print("데이터 수:", len(df_preprocessed))
print("결측치 수:", df_preprocessed['value'].isna().sum())
print("시간 간격:", df_preprocessed.index.to_series().diff().dropna().unique())

데이터 수: 11973
결측치 수: 0
시간 간격: <TimedeltaArray>
['0 days 00:05:00']
Length: 1, dtype: timedelta64[ns]
